In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score 

In [2]:
df = pd.read_excel("Aviation_KPIs_Dataset.xlsx")

In [3]:
df.head()

,Flight Number,Scheduled Departure Time,Actual Departure Time,Delay (Minutes),Aircraft Utilization (Hours/Day),Turnaround Time (Minutes),Load Factor (%),Fleet Availability (%),Maintenance Downtime (Hours),Fuel Efficiency (ASK),Revenue (USD),Operating Cost (USD),Net Profit Margin (%),Ancillary Revenue (USD),Debt-to-Equity Ratio,Revenue per ASK,Cost per ASK,Profit (USD)
0,FL885,2024-05-20 11:51:21.328,2024-07-10 02:38:53.731,50,12.36,115,79.18,96.24,9.21,4.15,10953.75,27847.59,17.90,1058.68,2.46,2639.46,6710.26,-16893.84
1,FL930,2024-01-23 06:56:22.686,2024-07-07 02:53:44.150,27,14.53,83,98.59,80.49,1.55,3.93,31597.25,1564.41,7.70,2941.32,2.89,8040.01,398.07,30032.84
2,FL478,2024-05-30 09:18:38.578,2024-12-05 01:00:54.473,108,10.73,87,67.44,97.90,2.06,4.26,13700.27,30494.88,5.84,1920.45,1.34,3216.03,7158.42,-16794.61
3,FL637,2024-08-15 05:21:46.814,2024-09-21 13:23:41.615,64,15.10,99,69.01,80.17,6.85,4.13,39913.01,13444.09,24.09,3167.79,0.60,9664.17,3255.23,26468.92
4,FL318,2024-07-25 15:29:58.467,2024-03-21 15:05:54.375,30,13.46,114,50.36,82.23,7.30,4.97,11531.48,34668.98,7.18,619.80,0.84,2320.22,6975.65,-23137.50


In [5]:
print(df.columns)

Index(['Scheduled Departure Time', 'Actual Departure Time', 'Delay (Minutes)',
       'Aircraft Utilization (Hours/Day)', 'Turnaround Time (Minutes)',
       'Load Factor (%)', 'Fleet Availability (%)',
       'Maintenance Downtime (Hours)', 'Fuel Efficiency (ASK)',
       'Revenue (USD)', 'Operating Cost (USD)', 'Net Profit Margin (%)',
       'Ancillary Revenue (USD)', 'Debt-to-Equity Ratio', 'Profit (USD)'],
      dtype='object')


In [7]:
# Feature Engineering
def feature_engineering(df):
    df['utilization_efficiency'] = df['Aircraft Utilization (Hours/Day)'] / (df['Turnaround Time (Minutes)'] + 1)
    df['profit_margin_ratio'] = df['Net Profit Margin (%)'] / 100
    df['cost_efficiency'] = df['Operating Cost (USD)'] / (df['Revenue (USD)'] + 1)
    df['fleet_utilization'] = df['Fleet Availability (%)'] * df['Aircraft Utilization (Hours/Day)']
    df['load_efficiency'] = df['Load Factor (%)'] * df['Fleet Availability (%)']
    return df

df = feature_engineering(df)

In [8]:

df['Scheduled_Month'] = df['Scheduled Departure Time'].dt.month

df['Actual_Month'] = df['Actual Departure Time'].dt.month


# Drop original datetime columns
df.drop(columns=['Scheduled Departure Time', 'Actual Departure Time'], inplace=True)


In [9]:
print(df.dtypes)  # Ensure no datetime columns remain


Delay (Minutes)                       int64
Aircraft Utilization (Hours/Day)    float64
Turnaround Time (Minutes)             int64
Load Factor (%)                     float64
Fleet Availability (%)              float64
Maintenance Downtime (Hours)        float64
Fuel Efficiency (ASK)               float64
Revenue (USD)                       float64
Operating Cost (USD)                float64
Net Profit Margin (%)               float64
Ancillary Revenue (USD)             float64
Debt-to-Equity Ratio                float64
Profit (USD)                        float64
utilization_efficiency              float64
profit_margin_ratio                 float64
cost_efficiency                     float64
fleet_utilization                   float64
load_efficiency                     float64
Scheduled_Month                       int32
Actual_Month                          int32
dtype: object


In [10]:
df.head()

,Delay (Minutes),Aircraft Utilization (Hours/Day),Turnaround Time (Minutes),Load Factor (%),Fleet Availability (%),Maintenance Downtime (Hours),Fuel Efficiency (ASK),Revenue (USD),Operating Cost (USD),Net Profit Margin (%),Ancillary Revenue (USD),Debt-to-Equity Ratio,Profit (USD),utilization_efficiency,profit_margin_ratio,cost_efficiency,fleet_utilization,load_efficiency,Scheduled_Month,Actual_Month
0,50,12.36,115,79.18,96.24,9.21,4.15,10953.75,27847.59,17.90,1058.68,2.46,-16893.84,0.106552,0.1790,2.542056,1189.5264,7620.2832,5,7
1,27,14.53,83,98.59,80.49,1.55,3.93,31597.25,1564.41,7.70,2941.32,2.89,30032.84,0.172976,0.0770,0.049509,1169.5197,7935.5091,1,7
2,108,10.73,87,67.44,97.90,2.06,4.26,13700.27,30494.88,5.84,1920.45,1.34,-16794.61,0.121932,0.0584,2.225697,1050.4670,6602.3760,5,12
3,64,15.10,99,69.01,80.17,6.85,4.13,39913.01,13444.09,24.09,3167.79,0.60,26468.92,0.151000,0.2409,0.336826,1210.5670,5532.5317,8,9
4,30,13.46,114,50.36,82.23,7.30,4.97,11531.48,34668.98,7.18,619.80,0.84,-23137.50,0.117043,0.0718,3.006203,1106.8158,4141.1028,7,3


In [11]:
season_mapping = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Fall", 10: "Fall", 11: "Fall"
}

df["Actual_Season"] = df["Actual_Month"].map(season_mapping)
df["Scheduled_Season"] = df["Scheduled_Month"].map(season_mapping)

In [12]:
from sklearn.preprocessing import LabelEncoder
le =  LabelEncoder()
# df["Scheduled_Season"] = df["Scheduled_Season"].astype("category").cat.codes
# df["Actual_Season"] = df["Actual_Month"].astype("category").cat.codes
df["Actual_Season"] = le.fit_transform(df["Actual_Season"])
df["Scheduled_Season"]  = le.fit_transform(df["Scheduled_Season"])

In [13]:
df.columns

Index(['Delay (Minutes)', 'Aircraft Utilization (Hours/Day)',
       'Turnaround Time (Minutes)', 'Load Factor (%)',
       'Fleet Availability (%)', 'Maintenance Downtime (Hours)',
       'Fuel Efficiency (ASK)', 'Revenue (USD)', 'Operating Cost (USD)',
       'Net Profit Margin (%)', 'Ancillary Revenue (USD)',
       'Debt-to-Equity Ratio', 'Profit (USD)', 'utilization_efficiency',
       'profit_margin_ratio', 'cost_efficiency', 'fleet_utilization',
       'load_efficiency', 'Scheduled_Month', 'Actual_Month', 'Actual_Season',
       'Scheduled_Season'],
      dtype='object')

In [14]:
df['Delay_Category'] = np.where(df['Delay (Minutes)'] < 30, 'Short',
                                np.where(df['Delay (Minutes)'] <= 60, 'Medium', 'Long'))

In [15]:
df['Delay_Category']  = le.fit_transform(df['Delay_Category'])

In [17]:
df1 = df.drop(columns = [ 'Scheduled_Month', 'Actual_Month','Delay (Minutes)'])

In [18]:
df1.head()

,Aircraft Utilization (Hours/Day),Turnaround Time (Minutes),Load Factor (%),Fleet Availability (%),Maintenance Downtime (Hours),Fuel Efficiency (ASK),Revenue (USD),Operating Cost (USD),Net Profit Margin (%),Ancillary Revenue (USD),Debt-to-Equity Ratio,Profit (USD),utilization_efficiency,profit_margin_ratio,cost_efficiency,fleet_utilization,load_efficiency,Actual_Season,Scheduled_Season,Delay_Category
0,12.36,115,79.18,96.24,9.21,4.15,10953.75,27847.59,17.90,1058.68,2.46,-16893.84,0.106552,0.1790,2.542056,1189.5264,7620.2832,2,1,1
1,14.53,83,98.59,80.49,1.55,3.93,31597.25,1564.41,7.70,2941.32,2.89,30032.84,0.172976,0.0770,0.049509,1169.5197,7935.5091,2,3,2
2,10.73,87,67.44,97.90,2.06,4.26,13700.27,30494.88,5.84,1920.45,1.34,-16794.61,0.121932,0.0584,2.225697,1050.4670,6602.3760,3,1,0
3,15.10,99,69.01,80.17,6.85,4.13,39913.01,13444.09,24.09,3167.79,0.60,26468.92,0.151000,0.2409,0.336826,1210.5670,5532.5317,0,2,0
4,13.46,114,50.36,82.23,7.30,4.97,11531.48,34668.98,7.18,619.80,0.84,-23137.50,0.117043,0.0718,3.006203,1106.8158,4141.1028,1,2,1


In [19]:
# Define features and target
X = df.drop(columns=["Profit (USD)", "Flight Number", "Revenue (USD)", "Operating Cost (USD)", "Revenue per ASK", "Cost per ASK"], errors="ignore")
y = df["Profit (USD)"]

# Handle missing values
X.fillna(X.median(numeric_only=True), inplace=True)

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train optimized Random Forest model
model = RandomForestRegressor(n_estimators=200, max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate performance
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2:.4f}")

Mean Absolute Error (MAE): 4300.230198493366
R² Score: 0.8951
